### Import required libraries for data acquisition

In [75]:
import pickle
from os import listdir, getcwd
from os.path import dirname, abspath, join

Load Data from `.pickle` files

In [77]:
data = {}
data_dir = abspath(dirname(getcwd()))
for f in listdir(data_dir):
    if '.pickle' in f:
        data_path = join(data_dir, f)
        with open(data_path, 'rb') as fp:
            data[data_path.split('/')[-1].split('.')[0]] = pickle.load(fp)

Convert list data to numpy array (`np.ndarray`)

In [79]:
import numpy as np

def data_to_numpy(data: list) -> np.array:
    arr = np.empty((len(data),len(data[0]), len(data[0][0])), np.float64)
    for i in range(len(data)):
        for j in range(len(data[0])):
            arr[i, j, :] = [data[i][j]['x'], data[i][j]['y'], data[i][j]['z']]
    return arr

In [80]:
# arr_l = 
arr_l = np.concatenate((data_to_numpy(data['l_s']), data_to_numpy(data['l_ns'])), axis=0)
arr_nl = np.concatenate((data_to_numpy(data['nl_s']), data_to_numpy(data['nl_ns'])), axis=0)

(3590, 478, 3)

(3348, 478, 3)

In [82]:
X = np.concatenate((arr_l, arr_nl), axis=0)
np.mean(X, axis=0).shape

(478, 3)

In [83]:
y = np.concatenate((np.ones(arr_l.shape[0]), np.zeros(arr_nl.shape[0])), axis=0)
y.shape

(6938,)

In [84]:
X.shape

(6938, 478, 3)

In [85]:
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# scalers = {}
# for i in range(X.shape[1]):
#     scalers[i] = StandardScaler()
#     X[:, i, :] = scalers[i].fit_transform(X[:, i, :])

# for i in range(X_test.shape[1]):
#     X_test[:, i, :] = scalers[i].transform(X_test[:, i, :])
# scaler = preprocessing.StandardScaler().fit(X)
# X = scaler.transform(X)
X = X - np.mean(X, axis=1).reshape(X.shape[0], 1, X.shape[2])
X = X.reshape(X.shape[0], -1)
X.shape
# X, y = shuffle(X, y, random_state=0)

(6938, 1434)

In [86]:
scalers[i].mean_

array([0.4503051 , 0.47122805, 0.01132533])

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [124]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=400, tol=0.001).fit(X_train, y_train)

from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()
# clf = KNeighborsClassifier(n_neighbors=3)
# clf = RadiusNeighborsClassifier(radius=0.1)
# clf = KDTree(X, leaf_size=2)
# clf = svm.SVC()
clf.fit(X_train, y_train)

MLPClassifier(max_iter=300, random_state=1, tol=0.001)

MLPClassifier(max_iter=400, random_state=1, tol=0.001)

In [125]:
y_pred = clf.predict(X_test)

In [126]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9729257641921397

In [127]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1035,   50],
       [  12, 1193]])